In [4]:
import pandas as pd
import numpy as np
from pathlib import Path

# Table of Contents

# References

move this later

1.  https://www.webmd.com/heart-disease/default.htm
2.  https://www.webmd.com/cholesterol-management/default.htm

# Background

Heart disease is "is the leading cause of death for men and women in the U.S."1  And, having high cholesterol is a major risk factor for heart disease.2  The goal of my project is to try to use NHANES survey data to predict total cholesterol level.  A model that predicts total cholesterol level from health information would be benificial to health apps.  If a fittness/health app can predict total cholesterol level, they can notify the user if their cholesterol level is high, promoting they see a doctor and hence reduce heart disease risk.

## note to self, may want to expand


# Data

The National Health and Nutrition Examination Survey is a yearly health exam and survey conducted by the CDC to gauge the overall health status of US citizens.  Each survey consists of multiple datasets in SAS format.

The data I chose to use comes from the 2013-2016 surveys.  Prior and posterior to those years, some variables changed which would make it difficult to combine the data.


The list of vaariables I chose to include for modeling can be seen below.

In [5]:
variables = pd.read_csv('variables.csv')
variables

,Dataset_Description,Variable_Name,Variable_Description
0,LabCholesterolTotal,SEQN,Participant Identification Number
1,LabCholesterolTotal,LBXTC,Total Cholesterol Level
2,FastingQuestionairre,PHAFSTHR,Hours Fasted for labs
3,Demographics,RIAGENDR,Gender
4,Demographics,RIDRETH3,Race
5,Demographics,RIDAGEYR,Age
6,BodyMeasuresExamination,BMXWT,Weight
7,BodyMeasuresExamination,BMXHT,Height
8,BodyMeasuresExamination,BMXBMI,BMI
9,BloodPressureExaminationData,BPXSY1,Systolic1




The reasons for chosing these variables are...

## notes
later want to say you only used first day nutrition
may want to change variables to only include recreation info and say why you only used recreational activity info

In [6]:
data_path = Path("../Data/df.csv")
if data_path.is_file():
    df = pd.read_csv("../Data/df.csv")
else:
    file_list = pd.read_csv('files.csv')
    years = file_list.Year.unique()
    dfs = [None]*2
    for i in range(len(years)):
        files = file_list[file_list.Year == years[i]]
        first = files[files.Notes=="read in first"]
        first_path = list(first["Path"])[0]
        first_type = list(first["Dataset_Description"])[0]
        first_variables = list(variables[variables.Dataset_Description==first_type]["Variable_Name"])
        df = pd.read_sas(first_path)
        df = df[first_variables].copy()
        others = files[files.Notes!="read in first"]
        for other_path in others["Path"]:
            other_type = list(others[others["Path"]==other_path]["Dataset_Description"])[0]
            temp_df = pd.read_sas(other_path)
            other_variables = ["SEQN"] + list(variables[variables.Dataset_Description==other_type]["Variable_Name"])
            temp_df = temp_df[other_variables].copy()
            df = df.merge(temp_df, on="SEQN", how="left")
        df.drop("SEQN", axis=1, inplace=True)
        dfs[i] = df.copy()
    df = pd.concat(dfs,ignore_index=True)
    df.to_csv("../Data/df.csv")
df.head()


,LBXTC,PHAFSTHR,RIAGENDR,RIDRETH3,RIDAGEYR,BMXWT,BMXHT,BMXBMI,BPXSY1,BPXSY2,...,MCQ160M,MCQ170M,SMQ681,PAQ650,PAQ655,PAD660,PAQ665,PAQ670,PAD675,KIQ022
0,173.0,3.0,1.0,3.0,62.0,94.8,184.5,27.8,128.0,124.0,...,2.0,NaN,2.0,2.0,NaN,NaN,1.0,6.0,30.0,NaN
1,265.0,12.0,1.0,3.0,53.0,90.4,171.4,30.8,146.0,140.0,...,2.0,NaN,1.0,2.0,NaN,NaN,2.0,NaN,NaN,NaN
2,229.0,10.0,1.0,3.0,78.0,83.4,170.1,28.8,138.0,132.0,...,1.0,2.0,2.0,2.0,NaN,NaN,2.0,NaN,NaN,NaN
3,174.0,2.0,2.0,3.0,56.0,109.8,160.9,42.4,132.0,134.0,...,2.0,NaN,2.0,2.0,NaN,NaN,2.0,NaN,NaN,NaN
4,204.0,10.0,2.0,4.0,42.0,55.2,164.9,20.3,100.0,114.0,...,2.0,NaN,1.0,2.0,NaN,NaN,2.0,NaN,NaN,NaN


## Initial data cleaning

As we are only interested in those that had their Cholesterol level checked, I'm filtering for those without cholesterol level missing.

In [32]:
df=df[df.LBXTC.notnull()].copy()
df.shape

(14880, 55)

Some questions on survey data get skipped given a certain response to a prior question.  For example, if a person has not been diagnosed with high blood pressure, then they will not be asked if they are taking medication for blood pressure.  Because of this there are more missing values than needed.  Before observing the amount of missing data, these missing values will be fixed.

### Blood Pressure/Cholesterol Questionnaire

### Other medical conditions Questionnaire

For those that have not been told they have a thyroid problem, they will still not have a thyroid problem. Accordingly I'm setting the value of MCQ170M to 2. 
For those that refused or don't know, I'm setting the original value to nan

In [35]:
df.loc[df.MCQ160M==2,'MCQ170M']=2
df.loc[df.MCQ160M==7,'MCQ160M']=np.nan
df.loc[df.MCQ160M==7,'MCQ170M']=np.nan
df.loc[df.MCQ160M==9,'MCQ160M']=np.nan
df.loc[df.MCQ160M==9,'MCQ170M']=np.nan
df.loc[df.MCQ170M==7,'MCQ170M']=np.nan
df.loc[df.MCQ170M==9,'MCQ170M']=np.nan

### Physical activity Questionnaire

### Alcohol Use Questionnaire

In [28]:
nulls = 100*df.isnull().sum()/df.shape[0]

In [36]:
nulls[nulls>30].sort_values(ascending=False)

BPXSY4     94.758065
BPXDI4     94.758065
MCQ170M    92.379032
BPQ100D    81.491935
ALQ110     78.629032
PAD660     74.395161
PAQ655     74.348118
KIQ022     64.099462
PAD675     63.165323
PAQ670     63.084677
ALQ120U    55.161290
ALQ130     55.127688
ALQ120Q    44.079301
BPQ090D    43.057796
dtype: float64